# Creación Data referente a JUZGADOS DE LETRAS Con competencia Penal

Se realiza un analisis mediante expresiones regulares del art. 28 al 40 del Código Organico de Tribunales (COT). En estos artículos se definen las comunas de Chile donde tendrán asiento este tipo de Tribunales y además su territorio de jurisdiccion. 

En estos artículos no estan explicitamente definidos como Tribunales Penales, pero si se distinguen los que son Civiles, por lo que se deben omitir estos y estudiar los demás casos, y verificar que no existan Juzgados de Garantías en dichas comunas.

In [1]:
import re
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
path_raw = "../data/external/cot"

# Articulo 28 al 40 COT ref: JUZGADOS DE LETRAS
### BUSCAR SOLO LOS QUE TENGAN COMPETENCIA PENAL

In [3]:
with open(f'{path_raw}/juzgadoletras.txt', 'r') as file:
    contenido_jl = ''
    for line in file.readlines():
        contenido_jl += line

Se construye una expresion regular para captar una lista con la información que se desea procesar, y así generar un dataframe.

In [4]:
regex_jl = r"(?:Art.\s[0-9\s(bis|ter|quáter)]+\.\s[\w\s]+\,(?P<Region>[\w\s\']+)\,[\w\s]+:\s+)|(?:(?:^[A]\.\-\s[\w\s]+:\s+)(?:(?:[\w\s\,]+[.|\;])+))|(?:(?:[\-\s][\w\s]+\:\s*)|(?:\s*(?:(?P<cant_juzg>[\w]+)(?:\s[J|j][a-z\,\s]+)(?P<JG>[\w\s]+)[\,|y]\s[\w]+\s(?P<Jueces>[\w]+)\s[a-z\s\,]+(?P<Competencia>[\w|\s|\,]+)[\;|\.]$)\s*))"
matches = re.findall(regex_jl, contenido_jl, re.MULTILINE)

#matches
# GENERA UNA ARREGLO DE LISTA DE 4 ELEMENTOS

# ELEMENTO 0 : REGIÓN
# ELEMENTO 1: Cantidad de juzgados ...
# Elemento 2: Asiento juzgado 
# ELEMENTO 3: Cantidad de Jueces
# ELEMENTO 4: Competencia (. es lo mismo de 3 0 7) o varias ciudades (separadas por , o y)

In [5]:
data_jl = []

for item in range(0,len(matches)):

    if matches[item][0] != '':
        region = f"REGION{matches[item][0].upper()}"
    else:    
        if matches[item][2] != '':
            ciudad = matches[item][2].upper() 
            juzgado = f"JUZGADO DE LETRAS Y GARANTIA {ciudad}"

        if matches[item][3] != '':
            if matches[item][3] == 'competencia':
                cantidad_jueces = 1
            else:
                cantidad_jueces = cleandata.transforma_numero(matches[item][3])
    
        if matches[item][4] != '':
            if matches[item][4] == 'a':
                competencia = ciudad
        
            else:  
                if matches[item][4] != '':
                    competencia = matches[item][4].upper()

            competencia = competencia.replace(" Y ",",")
            competencia = competencia.replace(" E ",",")
            competencia = competencia.replace(".","")
        
            comunas = competencia.split(",")
        
            for comuna in comunas:
                data_jl.append([region,juzgado,ciudad,cantidad_jueces,comuna.strip(),'LETRAS Y GARANTIA'])


df_juzgados_letras = pd.DataFrame(data_jl,columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO'])


In [6]:
cols = df_juzgados_letras.select_dtypes(include = ["object"]).columns
df_juzgados_letras[cols] = df_juzgados_letras[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 6/6 [00:00<00:00, 257.41it/s]


In [7]:
# Reset el index para realizar feather

df_juzgados_letras.reset_index(inplace = True)
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

# Guardamos dataset como archivo feather
df_juzgados_letras.to_feather(f'{path_interim}/generates_JuzgadosLetras.feather')
